In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
pi = np.pi

# Algebraic two grid method for Laplace equation on a square

The structure of the equation $\nabla^2 u(x) = 0$ on the grid is

$$\begin{pmatrix} 
T & I & 0 & 0 & 0 & \cdots & 0 & 0 & 0\\ 
I & T & I & 0 & 0 & \cdots & 0 & 0 & 0 \\
0 & I & T & I & 0 & \cdots & 0 & 0 & 0\\
0 & 0 & I & T & I & \cdots & 0 & 0 & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots &\\
0 & 0 & 0 & 0 & 0 &  0 & \cdots I & T & I \\
0 & 0 & 0 & 0 & 0 & 0 & \cdots 0 & I & T 
\end{pmatrix}\begin{pmatrix}f_{11} \\ f_{12} \\ f_{13} \\ \vdots \\ f_{N1} \\ \vdots \\ f_{NN} \end{pmatrix}=\vec{f},$$

where $T$ is a tridiagonal matrix of dimension $N_x \times N_y$.

$$\begin{pmatrix}
-4 & 1 & 0 & 0 & \cdots & 0 & 0 & 0\\
1 & -4 & 1 & 0 & \cdots & 0 & 0 & 0\\
0 & 1 & -4 & 1 & \cdots & 0 & 0 & 0\\
0 & 0 & -1 & 4 & \cdots & 0 & 0 & 0\\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots &  \\
0 & 0 & 0 & 0 & \cdots & 1 & -4 & 1 \\
0 & 0 & 0 & 0 & \cdots & 0 & -4 & 1 
\end{pmatrix}$$

and $I$ the identity matrix of the same dimension. The vector $\vec{f}$ contains the information of the boundary and has the following structure (we write it for $N=4$ for simplicity)

$$\vec{f}=\begin{pmatrix}-u_{01}-u_{10} \\ - u_{02} \\ -u_{03} \\ -u_{04}-u_{15} \\ -u_{20} \\ 0 \\ 0 \\ -u_{25}
\\-u_{30} \\ 0 \\ 0 \\ -u_{35} \\ - u_{40} - u_{51} \\ -u_{52} \\ -u_{53} \\ -u_{45} - u_{54}
\end{pmatrix}.$$

The easiest way of solving the problem is by computing $\vec{u} = A^{-1} \vec{f}$

### Gauss-Seidel method as a smoother
If we want to solve $A\vec{u}=\vec{f}$ we find the solution iteratively
$$u_i^{(k+1)}=\frac{1}{a_{ii}}\left(f_i - \sum_{j=i+1}^{n-1} a_{ij}u_j^{(k)} - \sum_{j=0}^{i-1}a_{ij}u_j^{(k+1)} \right) \quad i = 0, 1, \dots ,n-1 $$
If $i=0$ then we don't consider the last term

In [2]:
class Laplace:
    #--This class constructs the sets Nl, Sl, SlT and the transfer operators for one grid level--#
    #--it also has an implementation of the two-grid method--#
    def __init__(self,A,f,NVar,eps):
        self.f = f #Boundary conditions
        self.NVar = NVar #Dimension of the matrix A, equal to N*N
        self.A = A
        self.Ni = []
        self.Si = []
        self.SiT = []
        self.eps = eps #Free parameter used to create the strong couplings
        self.wij = 0 #Weights for interpolation
        self.Init = np.random.randint(self.NVar) #Init --> first C-Variable
        #This is the starting point for the coarsening. 
        #It can be random, in the end the result will be some 
        #kind of checkerboard for Laplace's equation. 
        self.UVar = set() #Undecided variables
        for i in range(self.NVar):
            self.UVar.add(i) #In the beginning all the variables are undecided        
        self.FVar = set() #Fine variables
        self.CVar = set() #Coarse variables 
        self.Lambda = np.zeros(self.NVar)
        self.ProlOp = None #Interpolation operator
        self.CoarseOp = None #Coarsening operator
        self.GalerkinOp = None #Galerkin operator
        self.u = np.zeros(self.NVar) #Initial guess for the solution.
        

    def get_Matrix(self):
        return self.A

    def get_UVar(self):
        return self.UVar
    def get_FVar(self):
        return self.FVar
    def get_CVar(self):
        return self.CVar

    def get_Ni(self):
        return self.Ni
    def get_Si(self):
        return self.Si
    def get_SiT(self):
        return self.SiT

    def get_Lambda(self):
        return self.Lambda
    def get_ProlOp(self):
        return self.ProlOp
    
    def NiSiSiT(self):
    #This computes the sets Ni, Si and SiT
    #Epsilon is used to define the strong couplings
        print('NVar',self.NVar) #Total number of variables
        #Ni
        for i in range(self.NVar):
            B = set() #Auxiliary list
            for j in range(self.NVar):
                if j != i and self.A[i,j] != 0:
                    B.add(j)
            self.Ni.append(B)
        #Si
        for i in range(self.NVar):
            B = set()
            for j in range(self.NVar):
                if self.A[i,j] < 0:
                    B.add(abs(self.A[i,j]))
            a_max = max(B)
            B = set()
            for j in range(self.NVar):
                if -self.A[i,j] >= self.eps*a_max and i != j:
                    B.add(j)  
            self.Si.append(B)
        #SiT
        for i in range(self.NVar):
            B = set()
            for j in range(self.NVar):
                if i in self.Si[j]:
                    B.add(j)
            self.SiT.append(B)

    def Lambda_i(self):
        self.Lambda = np.zeros(self.NVar) #We set to zero the values of the U-Variables
        for i in self.UVar:
            self.Lambda[i] = len(self.SiT[i].intersection(self.UVar)) + 2*len(self.SiT[i].intersection(self.FVar))

    def CoarseStep(self):   
        self.CVar.add(self.Init) 
        self.UVar.remove(self.Init)
        for i in self.SiT[self.Init]:
            self.FVar.add(i)
            if i in self.UVar:
                self.UVar.remove(i)
        self.Lambda_i() #We compute Lambda for all the U variables
        maxLamb = max(self.Lambda)
        NewPossibleC = [] #List with new possibe C-Variables
        if maxLamb != 0 and len(self.UVar) != 0: 
            for i in range(len(self.Lambda)):
                if self.Lambda[i] == maxLamb and i in self.UVar:
                    NewPossibleC.append(i)     
            self.Init = NewPossibleC[np.random.randint(len(NewPossibleC))] #Then we repeat the previous steps
        #If the maximum value of Lambda is zero, then there are no more undecided variables.
        else:
            self.Init = -1

    def coarsening(self):   
        while self.Init != -1:     
            self.CoarseStep()

    def Weight(self,i,j):
        #We compute the weight wij, i in F, j in Pi
        Pi = self.CVar.intersection(self.Si[i])
        SumA, SumB = 0, 0
        for k in self.Ni[i]:
            SumA += self.A[i,k]
        for k in Pi:
            SumB += self.A[i,k]
        self.wij = - SumA/SumB * self.A[i,j]/self.A[i,i] 

    def ConstructOperators(self): 
        #We construct the operators by mapping the basis vectors of the coarse grid with the interpolator
        self.ProlOp = np.zeros((self.NVar,len(self.CVar))) #Maps from coarse grid to fine grid
        cont = 0
        for cvar in self.CVar:
            image_vector = np.zeros(self.NVar)
            basis_vector = np.eye(1,self.NVar,cvar)[0] #The entries that correspond to F-variables are set to zero
            #We apply the operator
            for i in range(self.NVar):
                Pi = self.CVar.intersection(self.Si[i]) #Interpolation set
                if i in self.CVar:
                    image_vector[i] = basis_vector[i]
                else:
                    #i in FVar
                    for k in Pi:
                        self.Weight(i,k) #Compute the weight wik
                        image_vector[i] += self.wij*basis_vector[k]
            self.ProlOp[:,cont] = image_vector #We arrange the images in the columns to build the operator
            cont += 1
        self.CoarseOp = np.transpose(self.ProlOp)
        self.GalerkinOp = np.dot(np.dot(self.CoarseOp,self.A),self.ProlOp) 

    def GaussSeidel(self,nu,initial_guess):
        #f has the boundary conditions
        #nu is the number of iterations        
        self.u = initial_guess
        for k in range(nu):
            for i in range(self.NVar):
                SumA = np.dot(self.A[i,i+1:],self.u[i+1:])
                SumB = np.dot(self.A[i,:i],self.u[:i]) 
                self.u[i] = (self.f[i] - SumA - SumB) / self.A[i,i]
        return self.u

    def ExactInv(self):
        #Exact inversion of the matrix
        self.u = np.dot(np.linalg.inv(self.A),self.f)
        return self.u

    def set_up_phase(self):
        self.NiSiSiT() #Construct Ni Si and SiT
        self.coarsening() #Coarse the grid
        self.ConstructOperators() #Construct operators         

    def TwoGridMethod(self,nu1,nu2,initial_guess):
        #nu1 --> pre-smoothing,  nu2 --> post-smoothing
        #We have to call the set up phase outside the function       
        self.u = self.GaussSeidel(nu1,initial_guess) #Pre-smoothing
        errorC = np.dot(np.dot(np.linalg.inv(self.GalerkinOp),self.CoarseOp), self.f-np.dot(self.A,self.u) )
        self.u = self.u + np.dot(self.ProlOp,errorC) #Correction 
        self.u = self.GaussSeidel(nu2,self.u) #Post-smoothing
        return self.u

In [3]:
class Multigrid:
    def __init__(self,A,NVar,maxl,eps):
        #LEVEL l+1 IS COARSER THAN LEVEL l, i.e. IT HAS LESS VARIABLES
        self.NVar = NVar #Dimension of the matrix A
        self.eps = eps #Parameter used to create the strong couplings
        self.Al = [A] #We store the original matrix and Galerkin operator here
        self.ProlOpl = [] #We store the operators for each level
        #ProlOpl[l] has the operator that takes us from level l+1 to level l
        #Size(ProlOpl) is size(Al)-1
        self.maxl = maxl #Number of grid levels

    def get_Al(self):
        return self.Al
        
    def get_ProlOpl(self):
        return self.ProlOpl
        
    def set_up_phase(self):
        #We create the operators for all the grid levels
        for i in range(self.maxl):
            if  i != maxl-1:
                Nl = self.Al[i].shape[0] #Number of variables at level l
                Syst_l = Laplace(self.Al[i],np.zeros(Nl),Nl,self.eps) #I don't actually need the second argument, but
                #the way my program is written asks me for that thing to work.
                Syst_l.NiSiSiT() #Construct Ni, Si and SiT at the l-level
                Syst_l.coarsening() #Coarse the grid from l-level to l+1 -level
                Syst_l.ConstructOperators() #Construct operators from l-level to l+1 -level
                GalerkinOp = np.dot(
                np.dot( np.transpose(Syst_l.get_ProlOp()), Syst_l.get_Matrix() ), 
                Syst_l.get_ProlOp() ) 
                self.Al.append(GalerkinOp) #Store A at level l+1 
                self.ProlOpl.append(Syst_l.get_ProlOp()) #Store the interpolation operator from l+1 -level to l-level     

    def GaussSeidel(self,nu,fl,ul,level):
        #nu is the number of iterations
        Nl = len(ul)
        for k in range(nu):
            for i in range(Nl):
                SumA = np.dot(self.Al[level][i,i+1:],ul[i+1:])
                SumB = np.dot(self.Al[level][i,:i],ul[:i]) 
                ul[i] = (fl[i] - SumA - SumB) / self.Al[level][i,i]
        return ul

    def V_cycle(self,nu1,nu2,l,fl,ul):
        """
        Multi-grid with V-Cycle
        maxl is the finest level
        nu1: pre-smoothing
        nu2: post-smoothing
        We count l starting from zero
        """    
        if l == self.maxl - 1:
            return np.dot(np.linalg.inv(self.Al[l]),fl) #At this stage we only invert the matrix that's left  
        else:    
            ul = self.GaussSeidel(nu1,fl,ul,l) #Pre-smoothing. We don't want pre smoothing for the
            #exact solution
            fl_1 = np.dot( np.transpose(self.ProlOpl[l]),
                        fl-np.dot(self.Al[l],ul) ) #restriction
            ul_1 = np.zeros(self.Al[l+1].shape[0]) #Vector for the next level. 
            ul_1 = self.V_cycle(nu1,nu2,l+1,fl_1,ul_1)  
    
        ul = ul + np.dot(self.ProlOpl[l],ul_1)
        ul = self.GaussSeidel(nu2,fl,ul,l) #Pre-smoothing  #Post-smoothing
        return ul

In [4]:
N = 50
#Laplace matrix
A = np.zeros((N*N,N*N))
for i in range(N):
    for j in range(N):
        A[i+j*N,i+j*N] = -4 #Diagonal
for i in range(N-1):
    for j in range(N):            
        A[i+j*N,i+j*N+1] = 1 #Upper diagonal of the triangular submatrices
        A[i+j*N+1,i+j*N] = 1 #Lower diagonal of the triangular submatrices        
for i in range(N):
    for j in range(N-1):
        A[i+j*N  , i+j*N + N] = 1 #Identity upper matrix
        A[i+j*N + N , i+j*N ] = 1 #Identity lower matrix        
A = -A #We do this to be consistent with Stueben notes

#Boundary conditions vector
f = np.zeros(N*N)
L, H, h = 1, 1, 1/(N+1) 
x, y = np.zeros(N), np.zeros(N)
for i in range(N):
    x[i], y[i] = (i+1)*h*L, (i+1)*h*L
#Boundary at x = 0
fx = np.zeros(N) #f_0j
#Boundary at y = 0
fy = np.sin(2*pi*x/L) #fj0
#Boundary at y = H
fH = np.zeros(N) #fj4
#Boundary at x = L 
fL = np.zeros(N) #f4j
for i in range(N):
    f[i+(N-1)*i] = -fy[i] #Bottom boundary (y=0)
    f[i+(N-1)*i+ N-1 ] = -fH[i] #Top boundary (y=H)
for i in range(N):
    f[i] += -fx[i] #Left boundary (x=0)
    f[i+N*(N-1)] += - fL[i] #Right boundary (x=L)
f = -f #Consistency with Stueben notes
def AnalyticSolution(x,y):
    #Here x,y are numbers
    return np.sin(2*pi*x/L)*np.sinh(2*pi*(H-y)/L)/np.sinh(2*pi*H/L)

In [5]:
#np.random.seed(0)  #We use this seed here to exactly reproduce the two-grid method, but with the V-cycle algorithm
maxl, eps = 4, 0.25
#We measure the setup phase computing time separately
TIME = [] 
nmeas = 10 #We perform ten measurements to determine the computing time. 
for i in range(nmeas):
    start = time.time()
    AMG = Multigrid(A,N*N,maxl,eps) 
    AMG.set_up_phase()
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

NVar 2500
NVar 1250
NVar 312
NVar 2500
NVar 1250
NVar 314
NVar 2500
NVar 1250
NVar 312
NVar 2500
NVar 1250
NVar 316
NVar 2500
NVar 1250
NVar 314
NVar 2500
NVar 1250
NVar 313
NVar 2500
NVar 1250
NVar 312
NVar 2500
NVar 1250
NVar 311
NVar 2500
NVar 1250
NVar 313
NVar 2500
NVar 1250
NVar 313
Execution time 12.853164625167846 +- 0.04106448584789153 seconds


In [6]:
nu1, nu2, l = 5, 5, 0 #We start at l=0
u0 = np.zeros(N*N)
TIME = [] 
nmeas = 10
for i in range(nmeas):
    start = time.time()
    uAMG = AMG.V_cycle(nu1,nu2,l,f,u0) #We are not taking the set up phase into account
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

Execution time 0.13352444171905517 +- 0.0006774305049765493 seconds


In [7]:
eps = 0.25
#np.random.seed(0) 
TIME = [] 
nmeas = 10 #We perform ten measurements to determine the computing time. 
for i in range(nmeas):
    start = time.time()
    SystOfEqs = Laplace(A,f,N*N,eps)
    SystOfEqs.set_up_phase()
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

NVar 2500
NVar 2500
NVar 2500
NVar 2500
NVar 2500
NVar 2500
NVar 2500
NVar 2500
NVar 2500
NVar 2500
Execution time 10.65808618068695 +- 0.027415866411706104 seconds


In [8]:
TIME = [] 
nmeas = 10
for i in range(nmeas):
    start = time.time()
    uFD = SystOfEqs.ExactInv()
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

Execution time 0.18710529804229736 +- 0.0009886643932826482 seconds


In [9]:
u0 = np.zeros(N*N) #Initial guess
nu1, nu2 = 5, 5
TIME = [] 
nmeas = 10
for i in range(nmeas):
    start = time.time()
    uTGM = SystOfEqs.TwoGridMethod(nu1,nu2,u0)
    end = time.time()
    TIME.append(end-start)
TIME = np.array(TIME)
print('Execution time',np.mean(TIME),'+-',np.std(TIME)/np.sqrt(nmeas),'seconds')

Execution time 0.1480032682418823 +- 0.0020561317756586607 seconds


In [10]:
u0 = np.zeros(N*N) #Initial guess
nu = 10
uGS = SystOfEqs.GaussSeidel(nu,u0)

In [11]:
for i in range(N):
    for j in range(N):
        string = 'u{:>0}{:>0} | Analytical {:>8} | Exact Inversion {:>8}'+\
              ' | Gauss-Seidel {:>8} | Two-Grid {:>8}' +\
              ' | AMG {:>0} levels {:>8}'
        print(string.format(i+1,j+1,
        np.round(AnalyticSolution(x[i],y[j]),5),
        np.round(uFD[i*N+j],5), 
        np.round(uGS[i*N+j],5),
        np.round(uTGM[i*N+j],5),
        maxl, np.round(uAMG[i*N+j],5)
        ))     

u11 | Analytical  0.10864 | Exact Inversion  0.10866 | Gauss-Seidel  0.08808 | Two-Grid  0.10863 | AMG 4 levels  0.10854
u12 | Analytical  0.09605 | Exact Inversion  0.09608 | Gauss-Seidel  0.05951 | Two-Grid  0.09603 | AMG 4 levels  0.09585
u13 | Analytical  0.08492 | Exact Inversion  0.08496 | Gauss-Seidel  0.03803 | Two-Grid  0.08488 | AMG 4 levels  0.08462
u14 | Analytical  0.07507 | Exact Inversion  0.07512 | Gauss-Seidel  0.02309 | Two-Grid  0.07503 | AMG 4 levels   0.0747
u15 | Analytical  0.06637 | Exact Inversion  0.06642 | Gauss-Seidel  0.01337 | Two-Grid  0.06632 | AMG 4 levels  0.06593
u16 | Analytical  0.05868 | Exact Inversion  0.05873 | Gauss-Seidel  0.00742 | Two-Grid  0.05863 | AMG 4 levels  0.05819
u17 | Analytical  0.05188 | Exact Inversion  0.05193 | Gauss-Seidel  0.00396 | Two-Grid  0.05183 | AMG 4 levels  0.05135
u18 | Analytical  0.04586 | Exact Inversion  0.04592 | Gauss-Seidel  0.00204 | Two-Grid  0.04582 | AMG 4 levels  0.04531
u19 | Analytical  0.04055 | Exac